In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

db = 'jobs.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)

    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolate_level = None
        conn.execute(c)
        
def show_table():
    q= '''
    SELECT 
        name,
        type
    FROM sqlite_master
    WHERE type IN("table","view")
    '''
    return run_query(q)

show_table()

,name,type
0,recent_grads,table


In [2]:
first_5 = 'SELECT * FROM recent_grads LIMIT 5'
run_query(first_5)


,index,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,...,170,388,85,0.117241,75000,55000,90000,350,257,50
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,...,133,340,16,0.024096,73000,50000,105000,456,176,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,...,150,692,40,0.050125,70000,43000,80000,529,102,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972


In [3]:
# First 10 rows

first_10 = 'SELECT * FROM recent_grads LIMIT 10'
run_query(first_10)

,index,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,...,170,388,85,0.117241,75000,55000,90000,350,257,50
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,...,133,340,16,0.024096,73000,50000,105000,456,176,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,...,150,692,40,0.050125,70000,43000,80000,529,102,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972
5,5,6,2418,NUCLEAR ENGINEERING,Engineering,2573,17,2200,373,0.144967,...,264,1449,400,0.177226,65000,50000,102000,1142,657,244
6,6,7,6202,ACTUARIAL SCIENCE,Business,3777,51,832,960,0.535714,...,296,2482,308,0.095652,62000,53000,72000,1768,314,259
7,7,8,5001,ASTRONOMY AND ASTROPHYSICS,Physical Sciences,1792,10,2110,1667,0.441356,...,553,827,33,0.021167,62000,31500,109000,972,500,220
8,8,9,2414,MECHANICAL ENGINEERING,Engineering,91227,1029,12953,2105,0.139793,...,13101,54639,4650,0.057342,60000,48000,70000,52844,16384,3253
9,9,10,2408,ELECTRICAL ENGINEERING,Engineering,81527,631,8407,6548,0.437847,...,12695,41413,3895,0.059174,60000,45000,72000,45829,10874,3170


In [4]:
# Majors with ShareWomen > 0.5
sw_5 = 'SELECT Major FROM recent_grads WHERE ShareWomen >= 0.5 LIMIT 5'
run_query(sw_5)

,Major
0,ACTUARIAL SCIENCE
1,COMPUTER SCIENCE
2,ENVIRONMENTAL ENGINEERING
3,NURSING
4,INDUSTRIAL PRODUCTION TECHNOLOGIES


In [5]:
# Select majors with women in the minority.
min_women = '''SELECT Major,ShareWomen from recent_grads WHERE 
               ShareWomen <= 0.5 LIMIT 5'''
run_query(min_women)

,Major,ShareWomen
0,PETROLEUM ENGINEERING,0.120564
1,MINING AND MINERAL ENGINEERING,0.101852
2,METALLURGICAL ENGINEERING,0.153037
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,0.107313
4,CHEMICAL ENGINEERING,0.341631


In [6]:
# majors with female majority and median salary > 5000
f_med = '''SELECT Major, Major_Category,Median, ShareWomen FROM recent_grads
           WHERE ShareWomen >= 0.5 AND Median > 50000;'''
run_query(f_med)

,Major,Major_category,Median,ShareWomen
0,ACTUARIAL SCIENCE,Business,62000,0.535714
1,COMPUTER SCIENCE,Computers & Mathematics,53000,0.578766


In [7]:
# median >= 10000 OR unemployed people <= 1000
med_une = '''SELECT Major, Median, Unemployed FROM recent_grads
             WHERE Median >= 10000 OR Unemployed <= 1000
             LIMIT 20;'''

run_query(med_une)

,Major,Median,Unemployed
0,PETROLEUM ENGINEERING,110000,37
1,MINING AND MINERAL ENGINEERING,75000,85
2,METALLURGICAL ENGINEERING,73000,16
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,70000,40
4,CHEMICAL ENGINEERING,65000,1672
5,NUCLEAR ENGINEERING,65000,400
6,ACTUARIAL SCIENCE,62000,308
7,ASTRONOMY AND ASTROPHYSICS,62000,33
8,MECHANICAL ENGINEERING,60000,4650
9,ELECTRICAL ENGINEERING,60000,3895


In [8]:
# Engineering where sharewomen >= 0.5 , unemployment_rate < 0.051

e_sh_un = '''SELECT Major,Major_Category, ShareWomen,Unemployment_rate FROM recent_grads
             WHERE (Major_Category = 'Engineering') AND (ShareWomen >= 0.5 OR Unemployment_rate < 0.051);'''

run_query(e_sh_un)

,Major,Major_category,ShareWomen,Unemployment_rate
0,PETROLEUM ENGINEERING,Engineering,0.120564,0.018381
1,METALLURGICAL ENGINEERING,Engineering,0.153037,0.024096
2,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.107313,0.050125
3,MATERIALS SCIENCE,Engineering,0.310820,0.023043
4,ENGINEERING MECHANICS PHYSICS AND SCIENCE,Engineering,0.183985,0.006334
5,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.343473,0.042876
6,MATERIALS ENGINEERING AND MATERIALS SCIENCE,Engineering,0.292607,0.027789
7,ENVIRONMENTAL ENGINEERING,Engineering,0.558548,0.093589
8,INDUSTRIAL PRODUCTION TECHNOLOGIES,Engineering,0.750473,0.028308
9,ENGINEERING AND INDUSTRIAL MANAGEMENT,Engineering,0.174123,0.033652


In [9]:
# Order by Sharewomen in Descending order
desc_sw = '''SELECT Major,ShareWomen,Unemployment_rate from recent_grads
             WHERE ShareWomen > 0.3 AND Unemployment_rate < 0.1
             ORDER BY ShareWomen DESC'''

run_query(desc_sw)

,Major,ShareWomen,Unemployment_rate
0,EARLY CHILDHOOD EDUCATION,0.967998,0.040105
1,MATHEMATICS AND COMPUTER SCIENCE,0.927807,0.000000
2,ELEMENTARY EDUCATION,0.923745,0.046586
3,ANIMAL SCIENCES,0.910933,0.050862
4,PHYSIOLOGY,0.906677,0.069163
5,MISCELLANEOUS PSYCHOLOGY,0.905590,0.051908
6,HUMAN SERVICES AND COMMUNITY ORGANIZATION,0.904075,0.037819
7,NURSING,0.896019,0.044863
8,GEOSCIENCES,0.881294,0.024374
9,MASS MEDIA,0.877228,0.089837


In [10]:
# Engineering or Physical Science
eng_ps = '''SELECT Major_category,Major,Unemployment_rate FROM recent_grads
            WHERE (Major_category = 'Engineering' OR Major_Category = 'Physical Sciences')
            ORDER BY Unemployment_rate ASC'''

run_query(eng_ps)

,Major_category,Major,Unemployment_rate
0,Engineering,ENGINEERING MECHANICS PHYSICS AND SCIENCE,0.006334
1,Engineering,PETROLEUM ENGINEERING,0.018381
2,Physical Sciences,ASTRONOMY AND ASTROPHYSICS,0.021167
3,Physical Sciences,ATMOSPHERIC SCIENCES AND METEOROLOGY,0.022229
4,Engineering,MATERIALS SCIENCE,0.023043
5,Engineering,METALLURGICAL ENGINEERING,0.024096
6,Physical Sciences,GEOSCIENCES,0.024374
7,Engineering,MATERIALS ENGINEERING AND MATERIALS SCIENCE,0.027789
8,Engineering,INDUSTRIAL PRODUCTION TECHNOLOGIES,0.028308
9,Engineering,ENGINEERING AND INDUSTRIAL MANAGEMENT,0.033652


In [11]:
# COUNT
count = '''SELECT COUNT(Major) FROM recent_grads WHERE ShareWomen > 0.5'''

run_query(count)

,COUNT(Major)
0,97


In [12]:
# ENgineering Major with lowest median salary.
eng_min = '''SELECT Major,Major_category,MIN(Median) FROM recent_grads
             WHERE Major_category = 'Engineering';'''

run_query(eng_min)

,Major,Major_category,MIN(Median)
0,ARCHITECTURE,Engineering,40000


In [13]:
# Aggregating 
avg = '''SELECT AVG(Total), MIN(Men), MAX(Women) FROM recent_grads;'''

run_query(avg)

,AVG(Total),MIN(Men),MAX(Women)
0,39167.716763,119,307087


In [14]:
# number of roles, unemployment rate.
nroles = '''SELECT COUNT(Major) 'Number Of Majors', MAX(Unemployment_rate) 'Highest Unemployment Rate' FROM recent_grads;'''

run_query(nroles)

,Number Of Majors,Highest Unemployment Rate
0,173,0.177226


In [15]:
# UNIQUE values usiing DISTINCT()
unique = '''SELECT COUNT(DISTINCT(Major)) 'unique_major',COUNT(DISTINCT(Major_category)) 'unique_major_category',COUNT(DISTINCT(Major_code)) 'unique_major_code' FROM recent_grads;'''

run_query(unique)

,unique_major,unique_major_category,unique_major_code
0,173,16,173


In [16]:
# diff btw 25th and 75th percentile.
ptiles = '''SELECT Major,Major_category, (P75th - p25th) Quartile_Spread FROM recent_grads
            ORDER BY Quartile_Spread ASC
            LIMIT 20;
            '''

run_query(ptiles)

,Major,Major_category,Quartile_Spread
0,MILITARY TECHNOLOGIES,Industrial Arts & Consumer Services,0
1,SCHOOL STUDENT COUNSELING,Education,2000
2,LIBRARY SCIENCE,Education,2000
3,COURT REPORTING,Law & Public Policy,4000
4,PHARMACOLOGY,Biology & Life Science,5000
5,EDUCATIONAL ADMINISTRATION AND SUPERVISION,Education,6000
6,COUNSELING PSYCHOLOGY,Psychology & Social Work,6800
7,SPECIAL NEEDS EDUCATION,Education,10000
8,MATHEMATICS TEACHER EDUCATION,Education,10000
9,SOCIAL WORK,Psychology & Social Work,10000


In [17]:
# Using GROUPBY 
group = '''SELECT Major_category, AVG(ShareWomen) FROM recent_grads
           GROUP BY Major_category;'''

run_query(group)

,Major_category,AVG(ShareWomen)
0,Agriculture & Natural Resources,0.617938
1,Arts,0.561851
2,Biology & Life Science,0.584518
3,Business,0.405063
4,Communications & Journalism,0.643835
5,Computers & Mathematics,0.512752
6,Education,0.674986
7,Engineering,0.257158
8,Health,0.616857
9,Humanities & Liberal Arts,0.676193


In [18]:
# Percentage Employed.
percent_employed = '''SELECT Major_category,((AVG(Employed)/AVG(Total)) * 100)Share_Employed FROM recent_grads
                      GROUP BY Major_category;'''

run_query(percent_employed)

,Major_category,Share_Employed
0,Agriculture & Natural Resources,83.698628
1,Arts,80.674824
2,Biology & Life Science,66.715654
3,Business,83.596596
4,Communications & Journalism,84.222913
5,Computers & Mathematics,79.561082
6,Education,85.819015
7,Engineering,78.196669
8,Health,80.337413
9,Humanities & Liberal Arts,76.263827


In [19]:
# Filtering GROUP BY statement with HAVING
having = '''SELECT Major_category, AVG(low_wage_jobs)/AVG(Total) Share_Low_Wage FROM recent_grads
            GROUP BY Major_category
            HAVING Share_Low_Wage > 0.1;'''

run_query(having)

,Major_category,Share_Low_Wage
0,Arts,0.168331
1,Communications & Journalism,0.126324
2,Humanities & Liberal Arts,0.132087
3,Industrial Arts & Consumer Services,0.115713
4,Law & Public Policy,0.115685
5,Psychology & Social Work,0.116934
6,Social Science,0.102233


In [20]:
# ROUNDING decimals
rounding = '''SELECT ROUND(ShareWomen, 4), Major_category FROM recent_grads
              LIMIT 10;'''

run_query(rounding)

,"ROUND(ShareWomen, 4)",Major_category
0,0.1206,Engineering
1,0.1019,Engineering
2,0.1530,Engineering
3,0.1073,Engineering
4,0.3416,Engineering
5,0.1450,Engineering
6,0.5357,Business
7,0.4414,Physical Sciences
8,0.1398,Engineering
9,0.4378,Engineering


In [21]:
# ROUNDING MORE COMPLEX
round_2 = '''SELECT Major_category, ROUND(AVG(college_jobs)/AVG(Total), 3)share_degree_job FROM recent_grads
             GROUP BY Major_category
             HAVING share_degree_job < 0.3;'''

run_query(round_2)

,Major_category,share_degree_job
0,Agriculture & Natural Resources,0.248
1,Arts,0.265
2,Business,0.114
3,Communications & Journalism,0.220
4,Humanities & Liberal Arts,0.270
5,Industrial Arts & Consumer Services,0.249
6,Law & Public Policy,0.163
7,Social Science,0.215


In [22]:
# CASTING 
cast = '''SELECT Major_category,(CAST(Women AS FLOAT)/CAST(Total AS FLOAT)) SW FROM recent_grads
          GROUP BY Major_category
          ORDER BY SW;'''

run_query(cast)

,Major_category,SW
0,Law & Public Policy,0.002951
1,Engineering,0.113152
2,Arts,0.275868
3,Business,0.279172
4,Physical Sciences,0.468217
5,Communications & Journalism,0.506020
6,Biology & Life Science,0.637293
7,Health,0.717574
8,Psychology & Social Work,0.798746
9,Interdisciplinary,0.800911


In [23]:
# USING SUB QUERIES
sub = '''SELECT Major, Unemployment_rate FROM recent_grads
         WHERE Unemployment_rate < (SELECT AVG(Unemployment_rate) from recent_grads)
         ORDER BY Unemployment_rate ASC;'''

run_query(sub)

,Major,Unemployment_rate
0,MATHEMATICS AND COMPUTER SCIENCE,0.000000
1,BOTANY,0.000000
2,SOIL SCIENCE,0.000000
3,EDUCATIONAL ADMINISTRATION AND SUPERVISION,0.000000
4,ENGINEERING MECHANICS PHYSICS AND SCIENCE,0.006334
5,COURT REPORTING,0.011690
6,MATHEMATICS TEACHER EDUCATION,0.016203
7,PETROLEUM ENGINEERING,0.018381
8,GENERAL AGRICULTURE,0.019642
9,ASTRONOMY AND ASTROPHYSICS,0.021167


In [24]:
# SUBQUERY EXTENDED
sub_2 = '''SELECT CAST(COUNT(*) AS FLOAT)/ CAST((SELECT COUNT(*) FROM recent_grads) AS FLOAT) proportion_abv_avg FROM recent_grads 
           WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)'''

run_query(sub_2)

,proportion_abv_avg
0,0.526012


In [25]:
# 
m = '''
       SELECT Major_category FROM recent_grads
       GROUP BY Major_category
       ORDER BY SUM(Total) DESC
       LIMIT 5;'''

run_query(m)

,Major_category
0,Business
1,Humanities & Liberal Arts
2,Education
3,Engineering
4,Social Science


In [26]:
mm = '''SELECT Major, Major_category FROM recent_grads 
        WHERE Major_category IN
        (SELECT Major_category FROM recent_grads
        GROUP BY Major_category
        ORDER BY SUM(Total) DESC
        LIMIT 5);
       '''
run_query(mm)

,Major,Major_category
0,PETROLEUM ENGINEERING,Engineering
1,MINING AND MINERAL ENGINEERING,Engineering
2,METALLURGICAL ENGINEERING,Engineering
3,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering
4,CHEMICAL ENGINEERING,Engineering
5,NUCLEAR ENGINEERING,Engineering
6,ACTUARIAL SCIENCE,Business
7,MECHANICAL ENGINEERING,Engineering
8,ELECTRICAL ENGINEERING,Engineering
9,COMPUTER ENGINEERING,Engineering


> Lets say we are interested in understanding the ratio of the sample_size column to the total column.
 >* Computing this ratio for every major.
 >* Understanding which majors are above the average for this ratio.
 >* Understanding how many majors are above the average for this ratio.

In [27]:
# Average ratio for all the majors
avg_sub = '''SELECT AVG(CAST(Sample_size AS FLOAT) / CAST(Total AS FLOAT)) avg_ratio FROM recent_grads'''

run_query(avg_sub)

,avg_ratio
0,0.009086


In [28]:
com = '''SELECT Major,Major_category, (CAST(Sample_size AS FLOAT) / CAST(Total AS FLOAT)) Ratio FROM recent_grads
         WHERE Ratio >(SELECT AVG(CAST(Sample_size AS FLOAT)/CAST(Total AS FLOAT))avg_ratio FROM recent_grads)'''

run_query(com)

,Major,Major_category,Ratio
0,PETROLEUM ENGINEERING,Engineering,0.015391
1,MINING AND MINERAL ENGINEERING,Engineering,0.009259
2,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.012719
3,ACTUARIAL SCIENCE,Business,0.013503
4,MECHANICAL ENGINEERING,Engineering,0.011280
5,COMPUTER ENGINEERING,Engineering,0.009605
6,AEROSPACE ENGINEERING,Engineering,0.009762
7,INDUSTRIAL AND MANUFACTURING ENGINEERING,Engineering,0.009648
8,ARCHITECTURAL ENGINEERING,Engineering,0.009204
9,COURT REPORTING,Law & Public Policy,0.012195
